In [2]:
import numpy as np
import pandas as pd
import scipy.io as io
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# path = path to the 10Xv2 data directory containing genes.tsv, barcodes.tsv, and matrix.mtx files
# Note: need to import cell type annotations file manually since annotation files do not 
# have any set format between datasets. Set index names to cell barcode IDs in the annotation dataframe
# and have the celltype annotations as the first column.

class sctype:
    def __init__(self, path):
        self.genes = pd.read_csv(path + 'genes.tsv', sep = '\t', header = None).iloc[:,1]
        self.barcodes = pd.read_csv(path + 'barcodes.tsv', sep = '\t', header = None).iloc[:,0]
        expression = io.mmread(path + 'matrix.mtx') 
        data = pd.DataFrame.sparse.from_spmatrix(data = expression, index = self.genes, columns = self.barcodes)
        self.data = data.transpose()
    
#     labels input should be a dataframe with cell barcode IDs as indices
#     and celltype annotations in the first column.
    def split(self, labels, test_size, random_state):
        self.train_data, self.test_data, self.train_labels, self.test_labels = train_test_split(
            self.data, labels, test_size=test_size, random_state=random_state)
        
    def ann(self, epochs, batch_size, layers=[]):
        self.model = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out),)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        
        

In [8]:
path_data = '/Users/leealj/py_projects/biof509_final/zheng68k/filtered_matrices_mex/hg19/'
path_anno = '/Users/leealj/py_projects/biof509_final/zheng68k/'
labels = pd.read_csv(path_anno + '68k_pbmc_barcodes_annotation.tsv', sep = '\t').loc[:,["barcodes","celltype"]]
labels.set_index("barcodes", inplace = True)

In [44]:
import numpy as np
len(np.unique(labels))

11

In [ ]:
for epoch in range(epoch):
    running_loss = 0.0
    for i in

In [3]:
path_data = '/Users/leealj/py_projects/biof509_final/zheng68k/filtered_matrices_mex/hg19/'
zheng = sctype(path_data)

In [14]:
list(enumerate(labels.iloc[:,0], 0))

[(0, 'CD8+ Cytotoxic T'),
 (1, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (2, 'CD4+/CD45RO+ Memory'),
 (3, 'CD19+ B'),
 (4, 'CD4+/CD25 T Reg'),
 (5, 'CD4+/CD25 T Reg'),
 (6, 'CD4+/CD45RO+ Memory'),
 (7, 'CD8+ Cytotoxic T'),
 (8, 'CD8+ Cytotoxic T'),
 (9, 'CD56+ NK'),
 (10, 'CD4+/CD45RO+ Memory'),
 (11, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (12, 'CD4+/CD25 T Reg'),
 (13, 'CD8+ Cytotoxic T'),
 (14, 'CD8+ Cytotoxic T'),
 (15, 'CD8+ Cytotoxic T'),
 (16, 'CD4+/CD45RO+ Memory'),
 (17, 'CD8+ Cytotoxic T'),
 (18, 'CD19+ B'),
 (19, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (20, 'CD4+ T Helper2'),
 (21, 'CD56+ NK'),
 (22, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (23, 'CD19+ B'),
 (24, 'CD8+ Cytotoxic T'),
 (25, 'CD56+ NK'),
 (26, 'CD4+/CD25 T Reg'),
 (27, 'CD4+/CD25 T Reg'),
 (28, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (29, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (30, 'CD8+ Cytotoxic T'),
 (31, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (32, 'CD4+/CD45RA+/CD25- Naive T'),
 (33, 'CD8+/CD45RA+ Naive Cytotoxic'),
 (34, 'CD8+ Cytotoxic T'),
 (35,